In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [2]:
train_dir = 'D:/ComputerVision/Task4/Alzheimer_s Dataset/Alzheimer_s Dataset/train'  
test_dir = 'D:/ComputerVision/Task4/Alzheimer_s Dataset/Alzheimer_s Dataset/test'

In [3]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [4]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [5]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [6]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [7]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\badry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\badry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 13s 80ms/step


In [9]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step


In [10]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3196574	total: 144ms	remaining: 2m 23s
1:	learn: 1.2703438	total: 179ms	remaining: 1m 29s
2:	learn: 1.2256407	total: 210ms	remaining: 1m 9s
3:	learn: 1.1874951	total: 240ms	remaining: 59.8s
4:	learn: 1.1535276	total: 270ms	remaining: 53.8s
5:	learn: 1.1260974	total: 300ms	remaining: 49.7s
6:	learn: 1.1006441	total: 332ms	remaining: 47.1s
7:	learn: 1.0787094	total: 362ms	remaining: 44.9s
8:	learn: 1.0577698	total: 393ms	remaining: 43.3s
9:	learn: 1.0390492	total: 424ms	remaining: 42s
10:	learn: 1.0246470	total: 461ms	remaining: 41.4s
11:	learn: 1.0105536	total: 493ms	remaining: 40.6s
12:	learn: 0.9965360	total: 525ms	remaining: 39.9s
13:	learn: 0.9852053	total: 556ms	remaining: 39.2s
14:	learn: 0.9736747	total: 585ms	remaining: 38.4s
15:	learn: 0.9625390	total: 616ms	remaining: 37.9s
16:	learn: 0.9528832	total: 651ms	remaining: 37.6s
17:	learn: 0.9443569	total: 681ms	remaining: 37.1s
18:	learn: 0.9369972	total: 710ms	remaining: 36.7s
19:	learn: 0.9293038	total: 743ms	remaini

In [11]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [12]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 58.56%


In [13]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [14]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [15]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('D:/ComputerVision/Task4/Catboost/cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('D:/ComputerVision/Task4/Catboost/catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = "D:/ComputerVision/Task4/Alzheimer_s Dataset/Alzheimer_s Dataset/test/NonDemented/26 (88).jpg"
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
The image is classified as: NonDemented


C:\Users\badry\AppData\Local\Temp\ipykernel_8924\2977069490.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
